In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!ls

gdrive	sample_data


In [2]:
cd gdrive/MyDrive/UNCC/Spring22/Real-Time\ Machine\ Learning

/content/gdrive/MyDrive/UNCC/Spring22/Real-Time Machine Learning


In [3]:
cd Homeworks/Homework\ 3

/content/gdrive/MyDrive/UNCC/Spring22/Real-Time Machine Learning/Homeworks/Homework 3


In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [5]:
from torchvision import datasets, transforms
import time
from tqdm import tqdm

In [6]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

data_path = 'data/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=True)

dataloaders = {'train': train_loader, 
               'val': val_loader}

Files already downloaded and verified
Files already downloaded and verified


In [7]:
def training_loop(n_epochs, optimizer, model, loss_fn, dataloader, device):
    training_start_time = time.time()
    dur=[]
    

    train_accuracy_list=[]
    train_loss_list=[]
    val_accuracy_list=[]
    val_loss_list=[]
    best_acc=0
    best_epoch =0

    for epoch in range(1, n_epochs + 1):
        
        print('Epoch {}/{}'.format(epoch, n_epochs))
        print('-' * 10)
        for phase in ['train', 'val']:            
            t0 = time.time()

            correct = 0
            total = 0

            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0


            for i, data in enumerate(tqdm(dataloaders[phase])):
                
                imgs, labels = data
                imgs = imgs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
               
                #out = model(imgs.view(imgs.shape[0], -1))
                out = model(imgs)
                loss = loss_fn(out, labels.long())
                _, predicted = torch.max(out, dim=1)

                total += labels.shape[0]
                correct += int((predicted == labels).sum())

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                running_loss += float(loss)
                running_corrects += torch.sum(predicted == labels)
            t1= time.time() - t0
            dur.append(t1)
            epoch_loss = running_loss / len(dataloaders[phase].dataset) 
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            accuracy = (correct / total)

            if phase == 'val':
                
                val_accuracy_list.append(accuracy)
                val_loss_list.append(loss)
                dur.append(time.time() - t0)

            if phase == 'train':            

                train_accuracy_list.append(accuracy)
                train_loss_list.append(loss)  


            if phase == 'val' and accuracy > best_acc:
                best_acc = accuracy
                best_epoch = epoch

            #epoch_time= np.mean(dur)   
            #if n_epochs == 1 or n_epochs % 20 == 0: 
            print('{} Loss: {:.4f} Acc: {:.4f} Time: {}'.format(phase, loss,
                                                    accuracy, t1))
      
    print('Training finished, total training time {:.2f}s'.format(time.time() - training_start_time))
    print("Best Validation Accuracy: {:.4f} ".format(best_acc))
    print("Best Validation Accuracy achieved at Epoch: {} ".format(best_epoch))
    return [train_accuracy_list, train_loss_list, val_accuracy_list, val_loss_list]

In [8]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [9]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
    

In [10]:
print(torch.cuda.is_available())

True


In [12]:
model_2a = NetResDeep(n_chans1=32, n_blocks=10)
if torch.cuda.device_count() >= 1:
    model_2a = torch.nn.DataParallel(model_2a)
    device = torch.device("cuda")
    model_2a.to(device)
else:
    device='cpu'

In [13]:
numel_list = [p.numel() for p in model_2a.parameters()]
sum(numel_list), numel_list




(76074, [864, 32, 9216, 32, 32, 65536, 32, 320, 10])

In [18]:
learning_rate = 3e-3
n_epochs = 300
optimizer = optim.SGD(model_2a.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
test_2a = training_loop(
    n_epochs = n_epochs, 
    optimizer = optimizer,
    model = model_2a,
    loss_fn = loss_fn,   
    dataloader = dataloaders,
    device = device,)

Epoch 1/300
----------


100%|██████████| 782/782 [00:42<00:00, 18.45it/s]


train Loss: 0.7471 Acc: 0.8275 Time: 42.38063454627991


100%|██████████| 157/157 [00:04<00:00, 34.72it/s]


val Loss: 5.7019 Acc: 0.1886 Time: 4.53305721282959
Epoch 2/300
----------


...

100%|██████████| 782/782 [00:42<00:00, 18.31it/s]


train Loss: 0.0230 Acc: 0.9782 Time: 42.709004640579224


100%|██████████| 157/157 [00:04<00:00, 34.83it/s]


val Loss: 22.8468 Acc: 0.1327 Time: 4.516100645065308
Epoch 152/300
----------


 53%|█████▎    | 412/782 [00:22<00:20, 18.47it/s]

In [ ]:
t_range = torch.arange(1, 301)

fig = plt.figure(dpi=150)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot(t_range, torch.FloatTensor(test_2a[1]).cpu(), label='Training Loss')
plt.plot(t_range, torch.FloatTensor(test_2a[3]).cpu(), label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
t_range = torch.arange(1, 301)

fig = plt.figure(dpi=150)
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.plot(t_range, torch.FloatTensor(test_2a[0]).cpu(), label='Training Accuracy')
plt.plot(t_range, torch.FloatTensor(test_2a[2]).cpu(), label='Validation Accuracy')
plt.legend()
plt.show()